In [1]:
import os
import mykeys


In [2]:
mykeys.setOsEnv()

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.
mykey => set HUGGINGFACEHUB_API_TOKEN is done
mykey => set OPENAI_API_KEY is done
mykey => set LANGCHAIN_API_KEY is done
mykey => set LANGCHAIN_HUB_API_KEY is done
mykey => set GOOGLE_API_KEY is done
mykey => set GOOGLE_CSE_ID is done
mykey => set UPSTAGE_API_KEY is done
mykey => set COHERE_API_KEY is done
mykey => set JINA_API_KEY is done
mykey => set ANTHROPIC_API_KEY is done
mykey => set DEEPL_API_KEY is done
mykey => set TAVILY_API_KEY is done
mykey => set TOGETHER_API_KEY is done


In [6]:
import pandas as pd
import openai
import json
import os
import time
import re # 텍스트 파싱을 위한 정규식 라이브러리
import glob # (★★★ 신규) 파일 목록을 읽기 위해 추가

# --- 1. 설정 ---
INPUT_FILE = "20251030_synthetic_summaries_500.txt"
# (★★★ 신규) 개별 JSON 파일들을 저장할 디렉토리
PROFILE_DIR = "./user_profiles_split" 
OUTPUT_FILE = "user_profiles_for_hybrid_search.csv"
GPT_API_NAME = "gpt-4.1"

# (중요!) 테스트할 개수.
NUM_PROFILES_TO_PROCESS = 3 # (샘플링할 개수)

SLEEP_TIMER = 0.5 # API 속도 제한을 피하기 위한 대기 시간

# --- 2. OpenAI 클라이언트 설정 ---
try:
  client = openai.OpenAI()
  if not client.api_key:
    raise openai.OpenAIError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
except Exception as e:
  print(f"API 키 로드 오류: {e}")
  print("환경 변수(OPENAI_API_KEY)를 설정했는지 확인하세요.")
  exit()

# --- 3. LLM 프롬프트 정의 (요약문 -> JSON 추출) ---
METADATA_EXTRACTION_PROMPT = """
당신은 AI 데이터 추출기입니다.
사용자의 자기소개 요약문을 읽고, 레스토랑 데이터베이스를 쿼리하기 위한 6가지 '필터 조건'을 *JSON 객체*로 추출해야 합니다.

[추출할 6가지 메타데이터 키]
1.  `budget_range`: 사용자의 예산 ('저', '중', '고' 중 하나. "가성비", "저렴한" -> '저'. "적당한", "중간" -> '중'. "고급", "비싼" -> '고'. 언급 없으면 'N/A')
2.  `spicy_available`: 매운 음식을 선호하는지 여부 ('O', 'X' 중 하나. 언급 없으면 'N/A')
3.  `vegetarian_options`: 채식주의자인지 여부 ('O', 'X' 중 하나. '비건'도 'O'로 처리. 언급 없으면 'X')
4.  `main_ingredients_list`: 사용자가 *선호하는* 주재료. (콤마(,)로 구분된 *단일 문자열*. 예: "소고기,해산물,야채". 없으면 "N/A")
5.  `suitable_for`: 사용자의 여행 형태나 선호 분위기. (콤마(,)로 구분된 *단일 문자열*. 예: "혼자,조용한", "가족", "친구". 없으면 "N/A")
6.  `food_category`: 선호하는 음식 분류 ('한식', '중식', '일식', '양식', '동남아', '디저트류' 중 하나. 언급 없으면 'N/A')

[규칙]
- 응답은 *오직* 6개의 키를 가진 JSON 객체여야 합니다. (response_format={"type": "json_object"} 준수)
- 요약문에 명시적으로 언급되지 않은 키는 'N/A'로 처리하세요.
- `main_ingredients_list`는 *피하는 재료*가 아닌 *좋아하는 재료*만 포함합니다.
"""

# --- 4. 헬퍼 함수 정의 ---

def read_text_file(filepath):
  """텍스트 파일을 읽고 내용을 반환합니다."""
  try:
    with open(filepath, 'r', encoding='utf-8') as f:
      return f.read()
  except FileNotFoundError:
    print(f"[오류] 입력 파일을 찾을 수 없습니다: {filepath}")
    return None

def parse_profiles(text_content):
  """
  '---'로 구분된 텍스트를 파싱하여 ID, 이름, 요약문 리스트로 반환합니다.
  """
  print("텍스트 파일 파싱 중...")
  profiles = []
  
  # '---' 기준으로 텍스트 블록 분리
  blocks = text_content.split('---')
  
  # 정규식: "## 1. Soojin Kim (user_0001)"
  header_pattern = re.compile(r"## \d+\. (.*?) \((\S+)\)")
  
  for block in blocks:
    block = block.strip()
    if not block:
      continue
      
    lines = block.split('\n')
    header = lines[0].strip()
    summary = "\n".join(lines[1:]).strip() # 헤더 제외한 나머지
    
    match = header_pattern.search(header)
    
    if match and summary:
      name = match.group(1).strip()
      user_id = match.group(2).strip()
      profiles.append({
        "id": user_id,
        "name": name,
        "summary_text": summary
      })
  
  print(f"총 {len(profiles)}개의 프로필을 찾았습니다.")
  return profiles

def extract_metadata_from_summary(summary_text):
  """
  [LLM 2차 호출] 요약문에서 메타데이터 JSON을 추출합니다.
  """
  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=[
        {"role": "system", "content": METADATA_EXTRACTION_PROMPT},
        {"role": "user", "content": f"[사용자 요약문]\n{summary_text}"}
      ],
      response_format={"type": "json_object"},
      temperature=0.0 # 추출 작업이므로 창의성 0
    )
    
    # JSON 문자열을 파이썬 딕셔너리로 변환
    metadata_dict = json.loads(response.choices[0].message.content)
    return metadata_dict
  
  except Exception as e:
    print(f"  > [오류] LLM 메타데이터 추출 실패: {e}")
    # 오류 발생 시에도 후속 작업을 위해 빈 딕셔너리 반환
    return {}

# (★★★ 신규) 텍스트 파일을 개별 JSON 파일로 분할하는 전처리 함수
def preprocess_txt_to_individual_files(input_txt, output_dir):
  """
  (최초 1회 실행) 500개 텍스트 파일을 읽어 개별 .json 파일로 저장합니다.
  """
  print(f"'{output_dir}' 디렉토리 확인 중...")
  if os.path.exists(output_dir) and len(os.listdir(output_dir)) > 0:
    print(f"'{output_dir}'에 이미 파일이 분할되어 있으므로 전처리를 건너뜁니다.")
    return
    
  print(f"'{input_txt}' 파일을 읽어 개별 파일로 분할합니다...")
  os.makedirs(output_dir, exist_ok=True)
  
  raw_text = read_text_file(input_txt)
  if not raw_text:
    raise FileNotFoundError(f"입력 텍스트 파일({input_txt})을 찾을 수 없습니다.")
    
  all_profiles = parse_profiles(raw_text)
  
  for profile in all_profiles:
    user_id = profile['id']
    filename = os.path.join(output_dir, f"{user_id}.json")
    try:
      with open(filename, 'w', encoding='utf-8') as f:
        json.dump(profile, f, ensure_ascii=False, indent=2)
    except Exception as e:
      print(f"[오류] {filename} 저장 실패: {e}")
      
  print(f"총 {len(all_profiles)}개의 프로필을 '{output_dir}'에 개별 JSON 파일로 저장했습니다.")


# --- 5. 메인 스크립트 실행 (★★★ 수정됨 ★★★) ---
def main():
  
  # --- 0단계: 500개 텍스트 파일을 개별 JSON으로 분할 (최초 1회) ---
  try:
    preprocess_txt_to_individual_files(INPUT_FILE, PROFILE_DIR)
  except Exception as e:
    print(f"전처리 중 오류 발생: {e}")
    return

  # --- 1단계: 개별 파일 목록을 DataFrame으로 로드 ---
  print(f"'{PROFILE_DIR}'에서 프로필 파일 목록을 읽는 중...")
  
  # glob.glob: /path/to/user_0001.json ... 형태의 전체 경로 리스트를 가져옴
  profile_filepaths = glob.glob(os.path.join(PROFILE_DIR, "*.json"))
  
  if not profile_filepaths:
    print(f"[오류] '{PROFILE_DIR}'에서 .json 파일을 찾을 수 없습니다.")
    return

  all_files_df = pd.DataFrame({'filepath': profile_filepaths})
  print(f"총 {len(all_files_df)}개의 개별 프로필 파일 로드 완료.")

  # --- 2단계: Pandas Sample 적용 ---
  if NUM_PROFILES_TO_PROCESS > len(all_files_df):
    print(f"[경고] 요청된 개수({NUM_PROFILES_TO_PROCESS})가 전체 파일 수({len(all_files_df)})보다 많습니다. 전체를 처리합니다.")
    num_to_sample = len(all_files_df)
  else:
    num_to_sample = NUM_PROFILES_TO_PROCESS
    
  print(f"총 {len(all_files_df)}개 중 {num_to_sample}개를 *랜덤 샘플링*합니다...")
  profiles_to_process_df = all_files_df.sample(
    n=num_to_sample, 
    random_state=42 # 재현성을 위한 random_state
  )

  # --- 3단계: 메인 루프 (샘플링된 DF 순회) ---
  print(f"총 {len(profiles_to_process_df)}개 프로필에 대해 LLM 추출을 시작합니다...")
  final_data_list = []
  
  # .itertuples()를 사용해 DataFrame을 순회
  for i, row in enumerate(profiles_to_process_df.itertuples(index=False)):
    filepath = row.filepath
    
    try:
      # 개별 JSON 파일 읽기
      with open(filepath, 'r', encoding='utf-8') as f:
        profile = json.load(f)
    except Exception as e:
      print(f"[오류] {filepath} 파일 읽기 실패: {e}")
      continue
      
    user_id = profile['id']
    name = profile['name']
    summary = profile['summary_text']
    
    print(f"\n--- 작업 {i+1}/{len(profiles_to_process_df)} (ID: {user_id}, 파일: {os.path.basename(filepath)}) ---")
    print(f"  > [1/2] 요약문 전송 및 메타데이터 추출 중...")
    
    # [LLM 호출]
    metadata_dict = extract_metadata_from_summary(summary)
    
    if metadata_dict:
      print(f"  > [2/2] 추출 성공: {metadata_dict}")
      
      # CSV 저장을 위해 딕셔너리를 다시 JSON 문자열로 변환
      metadata_json_string = json.dumps(metadata_dict, ensure_ascii=False)
      
      final_data_list.append({
        "user_id": user_id,
        "name": name,
        "rag_query_text": summary, # (A) RAG 검색용 텍스트
        "filter_metadata_json": metadata_json_string # (B) 필터링용 메타데이터
      })
    else:
      print(f"  > [2/2] 추출 실패. 이 프로필을 건너뜁니다.")
      
    time.sleep(SLEEP_TIMER)

  # --- 4. CSV 파일로 저장 ---
  print(f"\n--- 모든 작업 완료. {len(final_data_list)}개 결과 저장 중 ---")
  
  if not final_data_list:
    print("저장할 데이터가 없습니다.")
    return
    
  df_final = pd.DataFrame(final_data_list)
  
  try:
    df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"성공! '{OUTPUT_FILE}' 파일이 생성되었습니다.")
    print("\n--- CSV 파일 샘플 ---")
    print(df_final.head())
  except Exception as e:
    print(f"[오류] CSV 파일 저장 실패: {e}")

if __name__ == "__main__":
  main()

'./user_profiles_split' 디렉토리 확인 중...
'20251030_synthetic_summaries_500.txt' 파일을 읽어 개별 파일로 분할합니다...
텍스트 파일 파싱 중...
총 500개의 프로필을 찾았습니다.
총 500개의 프로필을 './user_profiles_split'에 개별 JSON 파일로 저장했습니다.
'./user_profiles_split'에서 프로필 파일 목록을 읽는 중...
총 500개의 개별 프로필 파일 로드 완료.
총 500개 중 3개를 *랜덤 샘플링*합니다...
총 3개 프로필에 대해 LLM 추출을 시작합니다...

--- 작업 1/3 (ID: user_0475, 파일: user_0475.json) ---
  > [1/2] 요약문 전송 및 메타데이터 추출 중...
  > [2/2] 추출 성공: {'budget_range': '중', 'spicy_available': 'O', 'vegetarian_options': 'X', 'main_ingredients_list': '해산물,소고기,야채', 'suitable_for': '연인', 'food_category': '양식'}

--- 작업 2/3 (ID: user_0133, 파일: user_0133.json) ---
  > [1/2] 요약문 전송 및 메타데이터 추출 중...
  > [2/2] 추출 성공: {'budget_range': '중', 'spicy_available': 'X', 'vegetarian_options': 'X', 'main_ingredients_list': '닭고기,해산물,야채', 'suitable_for': '가족', 'food_category': 'N/A'}

--- 작업 3/3 (ID: user_0254, 파일: user_0254.json) ---
  > [1/2] 요약문 전송 및 메타데이터 추출 중...
  > [2/2] 추출 성공: {'budget_range': '중', 'spicy_available': 'X', 'vegetarian_o

In [7]:
import pandas as pd
import json
import os
import glob # 파일 목록을 검색하기 위해 import

# --- 1. 설정 ---
# (★★★) 개별 JSON 파일들이 저장된 디렉토리
PROFILE_DIR = "./user_profiles_split" 
# (★★★) 최종 저장할 CSV 파일 이름
OUTPUT_CSV = "user_profiles_combined.csv"

# --- 2. 메인 실행 함수 ---
def combine_json_files_to_csv():
  print(f"'{PROFILE_DIR}' 디렉토리에서 .json 파일 검색 중...")
  
  # PROFILE_DIR 안의 모든 .json 파일 경로를 리스트로 가져옴
  json_filepaths = glob.glob(os.path.join(PROFILE_DIR, "*.json"))
  
  if not json_filepaths:
    print(f"[오류] '{PROFILE_DIR}'에서 .json 파일을 찾을 수 없습니다.")
    print("이전 단계(preprocess_txt_to_individual_files)가 정상적으로 실행되었는지 확인하세요.")
    return

  print(f"총 {len(json_filepaths)}개의 .json 파일을 찾았습니다. DataFrame으로 변환을 시작합니다.")
  
  all_profiles_list = [] # 데이터를 저장할 리스트

  # 2. 모든 JSON 파일을 순회하며 읽기
  for filepath in json_filepaths:
    try:
      with open(filepath, 'r', encoding='utf-8') as f:
        # JSON 파일을 읽어 딕셔너리로 변환
        profile_data = json.load(f)
        all_profiles_list.append(profile_data)
        
    except Exception as e:
      print(f"[경고] {filepath} 파일 처리 중 오류 발생: {e}. 이 파일을 건너뜁니다.")
      
  if not all_profiles_list:
    print("[오류] 유효한 프로필 데이터를 하나도 읽지 못했습니다.")
    return

  # 3. 리스트를 Pandas DataFrame으로 변환
  print(f"총 {len(all_profiles_list)}개의 프로필을 DataFrame으로 변환 중...")
  df_combined = pd.DataFrame(all_profiles_list)
  
  # 4. CSV 파일로 저장
  try:
    df_combined.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(f"\n--- 작업 완료! ---")
    print(f"총 {len(df_combined)}개의 프로필이 '{OUTPUT_CSV}' 파일에 저장되었습니다.")
    print("\n--- CSV 파일 샘플 (상위 5개) ---")
    print(df_combined.head())
  except Exception as e:
    print(f"\n[치명적 오류] CSV 파일 저장 실패: {e}")

if __name__ == "__main__":
  combine_json_files_to_csv()

'./user_profiles_split' 디렉토리에서 .json 파일 검색 중...
총 500개의 .json 파일을 찾았습니다. DataFrame으로 변환을 시작합니다.
총 500개의 프로필을 DataFrame으로 변환 중...

--- 작업 완료! ---
총 500개의 프로필이 'user_profiles_combined.csv' 파일에 저장되었습니다.

--- CSV 파일 샘플 (상위 5개) ---
          id             name  \
0  user_0446  Giuseppe Romano   
1  user_0016    Chun Hei Wong   
2  user_0153      Lina Müller   
3  user_0295      Hazel Brown   
4  user_0041        Wei Zhang   

                                        summary_text  
0  안녕하세요! 저는 Giuseppe Romano입니다. 30대 이탈리아 남자로, 이번...  
1  안녕하세요! 저는 Chun Hei Wong입니다. 저는 20대 남자이고, 홍콩에서 ...  
2  안녕하세요! 저는 Lina Müller입니다. 독일에서 온 40대 여성이고요, 이번...  
3  안녕하세요! 저는 Hazel Brown입니다. 영국에서 온 20대 여성이고요, 이번...  
4  안녕하세요! 저는 Wei Zhang입니다. 저는 20대 남성이고 중국에서 왔어요. ...  


In [8]:
df_user = pd.read_csv('./user_profiles_combined.csv')

In [9]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            500 non-null    object
 1   name          500 non-null    object
 2   summary_text  500 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


In [10]:
df_user

id              name  \
0    user_0446   Giuseppe Romano   
1    user_0016     Chun Hei Wong   
2    user_0153       Lina Müller   
3    user_0295       Hazel Brown   
4    user_0041         Wei Zhang   
..         ...               ...   
495  user_0255      Matteo Rossi   
496  user_0310       Grace Ncube   
497  user_0081  Sebastian Müller   
498  user_0202   Sakura Yamamoto   
499  user_0347           Mei Lin   

                                          summary_text  
0    안녕하세요! 저는 Giuseppe Romano입니다. 30대 이탈리아 남자로, 이번...  
1    안녕하세요! 저는 Chun Hei Wong입니다. 저는 20대 남자이고, 홍콩에서 ...  
2    안녕하세요! 저는 Lina Müller입니다. 독일에서 온 40대 여성이고요, 이번...  
3    안녕하세요! 저는 Hazel Brown입니다. 영국에서 온 20대 여성이고요, 이번...  
4    안녕하세요! 저는 Wei Zhang입니다. 저는 20대 남성이고 중국에서 왔어요. ...  
..                                                 ...  
495  안녕하세요! 저는 Matteo Rossi입니다. 이탈리아에서 온 20대 남자예요. ...  
496  안녕하세요! 저는 Grace Ncube입니다. 저는 30대 여자이고, 남아프리카공화...  
497  안녕하세요! 저는 Sebastian Müller입니다. 저는 30대 남성이고, 독일...  
498  안녕하세요! 저는 Sakura Yamamoto입니다. 일본에서 온 20대 여자이고요...  
499  안녕하세요! 저는 Mei Lin입니다. 20대 싱가포르 여자고요, 이번에는 친구들이...  

[500 rows x 3 columns]

In [12]:
import pandas as pd
import openai
import json
import os
import time
# (re, glob는 더 이상 필요 없으므로 삭제)

# --- 1. 설정 ---
# (★★★ 수정됨) 500개 프로필이 들어있는 CSV 파일
INPUT_CSV = "user_profiles_combined.csv" 
OUTPUT_FILE = "user_profiles_for_hybrid_search.csv"
GPT_API_NAME = "gpt-4.1"

# (중요!) 테스트할 개수. 0으로 설정 시 전체 실행
NUM_PROFILES_TO_PROCESS = 50 # (샘플링할 개수)

SLEEP_TIMER = 0.5 # API 속도 제한을 피하기 위한 대기 시간

# --- 2. OpenAI 클라이언트 설정 ---
try:
  client = openai.OpenAI()
  if not client.api_key:
    raise openai.OpenAIError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
except Exception as e:
  print(f"API 키 로드 오류: {e}")
  print("환경 변수(OPENAI_API_KEY)를 설정했는지 확인하세요.")
  exit()

# --- 3. LLM 프롬프트 정의 (요약문 -> JSON 추출) ---
METADATA_EXTRACTION_PROMPT = """
당신은 AI 데이터 추출기입니다.
사용자의 자기소개 요약문을 읽고, 레스토랑 데이터베이스를 쿼리하기 위한 6가지 '필터 조건'을 *JSON 객체*로 추출해야 합니다.

[추출할 6가지 메타데이터 키]
1.  `budget_range`: 사용자의 예산 ('저', '중', '고' 중 하나. "가성비", "저렴한" -> '저'. "적당한", "중간" -> '중'. "고급", "비싼" -> '고'. 언급 없으면 'N/A')
2.  `spicy_available`: 매운 음식을 선호하는지 여부 ('O', 'X' 중 하나. 언급 없으면 'N/A')
3.  `vegetarian_options`: 채식주의자인지 여부 ('O', 'X' 중 하나. '비건'도 'O'로 처리. 언급 없으면 'X')
4.  `main_ingredients_list`: 사용자가 *선호하는* 주재료. (콤마(,)로 구분된 *단일 문자열*. 예: "소고기,해산물,야채". 없으면 "N/A")
5.  `suitable_for`: 사용자의 여행 형태나 선호 분위기. (콤마(,)로 구분된 *단일 문자열*. 예: "혼자,조용한", "가족", "친구". 없으면 "N/A")
6.  `food_category`: 선호하는 음식 분류 ('한식', '중식', '일식', '양식', '동남아', '디저트류' 중 하나. 언급 없으면 'N/A')

[규칙]
- 응답은 *오직* 6개의 키를 가진 JSON 객체여야 합니다. (response_format={"type": "json_object"} 준수)
- 요약문에 명시적으로 언급되지 않은 키는 'N/A'로 처리하세요.
- `main_ingredients_list`는 *피하는 재료*가 아닌 *좋아하는 재료*만 포함합니다.
"""

# --- 4. 헬퍼 함수 정의 ---

# (★★★ parse_profiles, read_text_file, preprocess... 함수 모두 삭제됨 ★★★)

def extract_metadata_from_summary(summary_text):
  """
  [LLM 호출] 요약문에서 메타데이터 JSON을 추출합니다.
  """
  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=[
        {"role": "system", "content": METADATA_EXTRACTION_PROMPT},
        {"role": "user", "content": f"[사용자 요약문]\n{summary_text}"}
      ],
      response_format={"type": "json_object"},
      temperature=0.0 # 추출 작업이므로 창의성 0
    )
    
    # JSON 문자열을 파이썬 딕셔너리로 변환
    metadata_dict = json.loads(response.choices[0].message.content)
    return metadata_dict
  
  except Exception as e:
    print(f"  > [오류] LLM 메타데이터 추출 실패: {e}")
    # 오류 발생 시에도 후속 작업을 위해 빈 딕셔너리 반환
    return {}


# --- 5. 메인 스크립트 실행 (★★★ 수정됨 ★★★) ---
def main():
  
  # --- 1단계: 500개 프로필이 담긴 CSV 로드 ---
  try:
    print(f"'{INPUT_CSV}' 파일에서 프로필 목록을 읽는 중...")
    all_profiles_df = pd.read_csv(INPUT_CSV)
  except FileNotFoundError:
    print(f"[오류] 입력 파일 '{INPUT_CSV}'를 찾을 수 없습니다.")
    return
  except Exception as e:
    print(f"[오류] CSV 파일 읽기 실패: {e}")
    return

  if not {'id', 'name', 'summary_text'}.issubset(all_profiles_df.columns):
    print("[오류] CSV 파일에 'id', 'name', 'summary_text' 컬럼이 모두 필요합니다.")
    return
    
  print(f"총 {len(all_profiles_df)}개의 개별 프로필 파일 로드 완료.")

  # --- 2단계: Pandas Sample 적용 ---
  if (
    NUM_PROFILES_TO_PROCESS > 0 and 
    NUM_PROFILES_TO_PROCESS < len(all_profiles_df)
  ):
    num_to_sample = NUM_PROFILES_TO_PROCESS
    print(f"총 {len(all_profiles_df)}개 중 {num_to_sample}개를 *랜덤 샘플링*합니다...")
    profiles_to_process_df = all_profiles_df.sample(
      n=num_to_sample, 
      random_state=42 # 재현성을 위한 random_state
    )
  else:
    print(f"총 {len(all_profiles_df)}개의 프로필 *전체*를 처리합니다.")
    profiles_to_process_df = all_profiles_df

  # --- 3단계: 메인 루프 (샘플링된 DF 순회) ---
  print(f"총 {len(profiles_to_process_df)}개 프로필에 대해 LLM 추출을 시작합니다...")
  final_data_list = []
  
  # .iterrows()를 사용해 DataFrame을 순회
  for i, (original_index, profile_row) in enumerate(profiles_to_process_df.iterrows()):
    
    user_id = profile_row['id']
    name = profile_row['name']
    summary = profile_row['summary_text']
    
    print(f"\n--- 작업 {i+1}/{len(profiles_to_process_df)} (ID: {user_id}, 이름: {name}) ---")
    print(f"  > [1/2] 요약문 전송 및 메타데이터 추출 중...")
    
    # [LLM 호출]
    metadata_dict = extract_metadata_from_summary(summary)
    
    if metadata_dict:
      print(f"  > [2/2] 추출 성공: {metadata_dict}")
      
      # CSV 저장을 위해 딕셔너리를 다시 JSON 문자열로 변환
      metadata_json_string = json.dumps(metadata_dict, ensure_ascii=False)
      
      final_data_list.append({
        "user_id": user_id,
        "name": name,
        "rag_query_text": summary, # (A) RAG 검색용 텍스트
        "filter_metadata_json": metadata_json_string # (B) 필터링용 메타데이터
      })
    else:
      print(f"  > [2/2] 추출 실패. 이 프로필을 건너뜁니다.")
      
    time.sleep(SLEEP_TIMER)

  # --- 4. CSV 파일로 저장 ---
  print(f"\n--- 모든 작업 완료. {len(final_data_list)}개 결과 저장 중 ---")
  
  if not final_data_list:
    print("저장할 데이터가 없습니다.")
    return
    
  df_final = pd.DataFrame(final_data_list)
  
  try:
    df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"성공! '{OUTPUT_FILE}' 파일이 생성되었습니다.")
    print("\n--- CSV 파일 샘플 ---")
    print(df_final.head())
  except Exception as e:
    print(f"[오류] CSV 파일 저장 실패: {e}")

if __name__ == "__main__":
  main()

'user_profiles_combined.csv' 파일에서 프로필 목록을 읽는 중...
총 500개의 개별 프로필 파일 로드 완료.
총 500개 중 50개를 *랜덤 샘플링*합니다...
총 50개 프로필에 대해 LLM 추출을 시작합니다...

--- 작업 1/50 (ID: user_0475, 이름: Emily Walker) ---
  > [1/2] 요약문 전송 및 메타데이터 추출 중...
  > [2/2] 추출 성공: {'budget_range': '중', 'spicy_available': 'O', 'vegetarian_options': 'X', 'main_ingredients_list': '해산물,소고기,야채', 'suitable_for': '연인', 'food_category': '양식'}

--- 작업 2/50 (ID: user_0133, 이름: Ahmed Hassan) ---
  > [1/2] 요약문 전송 및 메타데이터 추출 중...
  > [2/2] 추출 성공: {'budget_range': '중', 'spicy_available': 'X', 'vegetarian_options': 'X', 'main_ingredients_list': '닭고기,해산물,야채', 'suitable_for': '가족', 'food_category': 'N/A'}

--- 작업 3/50 (ID: user_0254, 이름: Aiko Tanaka) ---
  > [1/2] 요약문 전송 및 메타데이터 추출 중...
  > [2/2] 추출 성공: {'budget_range': '중', 'spicy_available': 'X', 'vegetarian_options': 'X', 'main_ingredients_list': '생선,해산물,야채', 'suitable_for': '친구', 'food_category': '일식'}

--- 작업 4/50 (ID: user_0102, 이름: Lucas Müller) ---
  > [1/2] 요약문 전송 및 메타데이터 추출 중...
  > [2/2